In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeCusco
from qiskit import transpile

import numpy as np

In [2]:
# Load saved credentials
service = QiskitRuntimeService()

## real ibm device

matlab fsolve 탑재 -> scipy module 과 matlab fsolve가 사용하는 solver가 달라서 완전 일치하지 않음.

backend_properties와 solved parameters 비교 기능 -> 두 값의 차이가 너무 커서... 더 고민 해야할 듯...

In [6]:
from datetime import datetime

real_backend = service.least_busy()
#pass_manager = generate_preset_pass_manager(optimization_level=0, backend=backend, initial_layout = [43,42,34,44,41,24,45]) # initial_layout=[], layout_method='trivial'

backend_prop = real_backend.properties().to_dict()
print("Backend name:", backend_prop['backend_name'])

Backend name: ibm_torino


In [6]:
test = [0,1,2,3,4]
print(test[1:1])
print(test[1:2])
print(test[1:3])

[]
[1]
[1, 2]


In [8]:
initial_layout = [63,62,73,64,61,82,65]

shots = 5000

RR = 30

num_target = 1

print("chosen device:",backend_prop['backend_name'])
print("chosen qubits:", initial_layout[0:num_target+1])
print("#shots:",shots,"#repetition:",RR)

#distilled_results = np.zeros((RR,M*N,3))
real_all_counts = [{'pad':0}]

for target in initial_layout[1:num_target+1]:
    pass_manager = generate_preset_pass_manager(optimization_level=0, backend=real_backend, initial_layout = [initial_layout[0], target])
    for rr in range(RR):

        qr = QuantumRegister(2)
        
        qc = QuantumCircuit(qr)
        
        qc.cx(0,1)
        qc.measure_all()
        
        isa_circuit = pass_manager.run(qc)
        
        sampler = Sampler(real_backend)
        job = sampler.run([isa_circuit], shots = shots)
        result = job.result()
        print(result)
        #print(result['__value__']['data'].meas.get_counts())
        counts = result['__value__']['data'].meas.get_counts()
        if '00' not in counts.keys():
            counts['00'] = 0
        if '01' not in counts.keys():
            counts['01'] = 0
        if '10' not in counts.keys():
            counts['10'] = 0
        if '11' not in counts.keys():
            counts['11'] = 0
        
        real_all_counts.append(counts)
        
        print(target,rr, 'counts: ', counts)
        
real_all_counts = real_all_counts[1:]
for cc in real_all_counts:
    print(cc)

chosen device: ibm_torino
chosen qubits: [63, 62]
#shots: 5000 #repetition: 30


UnboundLocalError: cannot access local variable 'frequency' where it is not associated with a value

### statistical analysis

In [8]:
#for cc in sim_all_counts:
#    print(cc['00']/shots,cc['10']/shots,cc['01']/shots,cc['11']/shots)

real_prob_data = {}

for numt in range(num_target):
    probs = {'00':0,'01':0,'10':0,'11':0}
    for rr in range(RR):
        for outcome in probs.keys():
            probs[outcome] += real_all_counts[numt*RR + rr][outcome[-1::-1]] / RR / shots
    #real_prob_data.append(probs)
    real_prob_data[initial_layout[numt+1]] = probs
        
    
print("big-endian encoding") # IBM qn ... q1 q0 => q0 q1 ... qn
for qubit, freq in real_prob_data.items():
    print(qubit, freq)

big-endian encoding
61 {'00': 0.9835333333333335, '01': 0.005913333333333332, '10': 0.006973333333333332, '11': 0.003579999999999999}
72 {'00': 0.9176600000000001, '01': 0.07110666666666667, '10': 0.0077800000000000005, '11': 0.003453333333333333}
63 {'00': 0.9565999999999999, '01': 0.03078, '10': 0.007346666666666665, '11': 0.005273333333333332}


### Backend Properties

In [5]:
print("Backend name:", backend_prop['backend_name'])
print("Last calibrated date:", backend_prop['last_update_date'])
for qq in range(num_target+1):
    print("========================================================================================")
    print("qubit",initial_layout[qq],"| readout_error (%):", backend_prop['qubits'][initial_layout[qq]][4]['value'])
    print("qubit",backend_prop['gates'][initial_layout[qq]]['qubits'][0],"| id_error (%):", backend_prop['gates'][initial_layout[qq]]['parameters'][0]['value'])
    print("qubit",backend_prop['gates'][initial_layout[qq]+127]['qubits'][0],"| rz_error (%):", backend_prop['gates'][initial_layout[qq]+127]['parameters'][0]['value'])
    print("qubit",backend_prop['gates'][initial_layout[qq]+127*2]['qubits'][0],"| sx_error (%):", backend_prop['gates'][initial_layout[qq]+127*2]['parameters'][0]['value'])
    print("qubit",backend_prop['gates'][initial_layout[qq]+127*2+127]['qubits'][0],"| x_error (%):", backend_prop['gates'][initial_layout[qq]+127*2+127]['parameters'][0]['value'])

    if qq > 0:
        for ecr_spec in backend_prop['gates'][127*4:]:
            if (initial_layout[0] in ecr_spec['qubits']) and (initial_layout[qq] in ecr_spec['qubits']):
                print("qubits", ecr_spec['qubits'],"| ecr_error (%):",ecr_spec['parameters'][0]['value'])
                
        

Backend name: ibm_brisbane
Last calibrated date: 2024-12-03 09:38:52+09:00
qubit 62 | readout_error (%): 0.013700000000000045
qubit 62 | id_error (%): 0.0002842175674138187
qubit 62 | rz_error (%): 0
qubit 62 | sx_error (%): 0.0002842175674138187
qubit 62 | x_error (%): 0.0002842175674138187
qubit 61 | readout_error (%): 0.0129999999999999
qubit 61 | id_error (%): 0.0005341108776756973
qubit 61 | rz_error (%): 0
qubit 61 | sx_error (%): 0.0005341108776756973
qubit 61 | x_error (%): 0.0005341108776756973
qubits [62, 61] | ecr_error (%): 0.006853695366647483


## Solve equations to get error parameters

The probabilities from measurement experiments can be seen as functions of three error parameters $f,q$ and $\epsilon$. So, we can solve the series of equations for known probabilities $p(ij)$.

$$p(00) = (1-\epsilon)\left[f^2 (1-q)^2 + (1-f)^2 q^2 + f(1-f)(1-q)q  + (1-f)^2 q(1-q) + (1-f)fq^2\right] + \epsilon/4 $$
$$p(01) = (1-\epsilon)\left[f^2 (1-q)q + (1-f)^2 q^2 + f(1-f)(1-q)^2  + (1-f)^2 q^2 + (1-f)fq(1-q)\right] + \epsilon/4 $$
$$p(10) = (1-\epsilon)\left[f^2 q(1-q) + (1-f)^2 (1-q)q + f(1-f)q^2  + (1-f)^2 (1-q)^2 + (1-f)f(1-q)q\right] + \epsilon/4 $$
$$p(11) = (1-\epsilon)\left[f^2 q^2 + (1-f)^2 (1-q)^2 + f(1-f) q(1-q)  + (1-f)^2 (1-q)q + (1-f)f(1-q)^2\right] + \epsilon/4 $$

In [63]:
from scipy.optimize import root

def equations(X,P):
    return [(1-X[2])*((X[0]**2)*(X[1]**2)+((1-X[0])**2)*((1-X[1])**2)+X[0]*(1-X[0])*X[1]*(1-X[1])+((1-X[0])**2)*(1-X[1])*X[1]+(1-X[0])*X[0]*((1-X[1])**2))+X[2]/4-P['11'],
        (1-X[2])*((X[0]**2)*(1-X[1])*X[1]+((1-X[0])**2)*(1-X[1])*X[1]+X[0]*(1-X[0])*(X[1]**2)+((1-X[0])**2)*((1-X[1])**2)+(1-X[0])*X[0]*(1-X[1])*X[1])+X[2]/4-P['10'],
        (1-X[2])*((X[0]**2)*(1-X[1])*X[1]+((1-X[0])**2)*(X[1]**2)+X[0]*(1-X[0])*((1-X[1])**2)+((1-X[0])**2)*(X[1]**2)+(1-X[0])*X[0]*(1-X[1])*X[1])+X[2]/4-P['01']]#,
        #(1-X[2])*((X[0]**2)*((1-X[1])**2)+((1-X[0])**2)*(X[1]**2)+X[0]*(1-X[0])*(1-X[1])*X[1]+((1-X[0])**2)*X[1]*(1-X[1])+(1-X[0])*X[0]*(X[1]**2))+X[2]/4-P['00']]

for qubit, freq in real_prob_data.items():
    #probs = [freq['00'],freq['01'],freq['10'],freq['11']]
    print(qubit, freq)
    solution = root(equations, [0,0,0], args=freq)
    print("(1-f,q,e) =", solution.x)


61 {'00': 0.9835333333333335, '01': 0.005913333333333332, '10': 0.006973333333333332, '11': 0.003579999999999999}
(1-f,q,e) = [ 0.00796449 -0.99449949  1.14254831]
72 {'00': 0.9176600000000001, '01': 0.07110666666666667, '10': 0.0077800000000000005, '11': 0.003453333333333333}
(1-f,q,e) = [ 0.01031037 -0.83577618  1.15734448]
63 {'00': 0.9565999999999999, '01': 0.03078, '10': 0.007346666666666665, '11': 0.005273333333333332}
(1-f,q,e) = [0.11398513 0.49085597 2.1982331 ]


### Results from MATLAB.fsolve

System qubit = 62,

target qubit = 61: $(1-f,q,\epsilon) = (-0.0011, 0.0024, 0.0185)$

target qubit = 72: $(1-f,q,\epsilon) = (0.0648, 0.0692, -0.3035)$

target qubit = 63: $(1-f,q,\epsilon) = (0.0240, 0.0262, -0.0805)$